# 📘 Pre-entrega

**Nombre del alumno:** Sanvido Gonzalo sebastian


## 🧩 Etapa 1: Recopilación y Preparación de Datos
**Objetivo:** Demostrar habilidades en Python, familiaridad con el entorno de trabajo y conocimientos básicos sobre manipulación de datos.

### 1️⃣ Carga de datos

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Montar la unidad
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# Verificar que los archivos csv se encuentren en la carpeta datasets
import os
os.listdir("/content/drive/MyDrive/datasets")

In [ ]:
# Definimos las rutas de los datasets.

ruta_ventas = "/content/drive/MyDrive/datasets/ventas.csv"
ruta_clientes = "/content/drive/MyDrive/datasets/clientes.csv"
ruta_marketing = "/content/drive/MyDrive/datasets/marketing.csv"

# Cargamos los CSV como DataFrames.
ventas = pd.read_csv(ruta_ventas)
clientes = pd.read_csv(ruta_clientes)
marketing = pd.read_csv(ruta_marketing)

# Validamos formas para comprobar que se cargaron correctamente.
print("ventas.shape ->", ventas.shape)
print("clientes.shape ->", clientes.shape)
print("marketing.shape ->", marketing.shape)

# Mostramos las primeras filas de cada dataset para corroborar estructura de columnas.
display(ventas.head(3))
display(clientes.head(3))
display(marketing.head(3))

### 2️⃣ Análisis exploratorio inicial

In [ ]:
def eda(df, nombre):
    print(f"=== {nombre} ===")
    print(f"Dimensiones del DataFrame: Filas={df.shape[0]}, Columnas={df.shape[1]}\n")

    print("info:")
    df.info()
    print("\n")

    print("columnas:", list(df.columns))
    print("dtypes:")
    print(df.dtypes)

    print("\nNulos por columna:")
    print(df.isna().sum())

    print("\nNulos por columna en %:")
    print( (((df.isna().sum() / df.shape[0]) * 100 ).round(2)).astype('str') +" %"  )

    print("\nDuplicados totales:")
    print(df.duplicated( keep=False ).sum())

    print("\nPrimeras filas:")
    display(df.head(3))

    print("\nDescribe (numérico):")
    display(df.describe(include='number'))
    print("-"*100)

    print("\nDescribe (categórico):")
    display(df.describe(include='object'))
    print("-"*100)

In [ ]:
eda(ventas, "ventas")

In [ ]:
eda(clientes, "clientes")

In [ ]:
eda(marketing, "marketing")

### 3️⃣ Calidad de los datos

In [ ]:

def calidad(df, nombre, clave=None):
    """
    Analiza la calidad del DataFrame:
      - Muestra cantidad de nulos por columna.
      - Cuenta filas duplicadas completas.
      - Si se indica una clave, muestra los valores duplicados más frecuentes.
    Parámetros:
      df: DataFrame de pandas que se analizará.
      nombre: texto descriptivo del DataFrame (ejemplo: 'VENTAS').
      clave: (opcional) nombre de la columna para buscar duplicados específicos.
    """

    # -------------------------------------------------
    # Mostrar título descriptivo con el nombre del DF
    # -------------------------------------------------
    print(f"### {nombre}")

    # -------------------------------------------------
    # Mostrar cantidad de valores nulos por columna
    # -------------------------------------------------
    # df.isna() devuelve un DataFrame booleano con True donde hay NaN.
    # .sum() cuenta los True (o sea, los nulos) por columna.
    # .to_frame("nulos") convierte el resultado en un DataFrame con una columna llamada 'nulos'.
    display(df.isna().sum().to_frame("nulos"))

    # -------------------------------------------------
    # Contar filas duplicadas completas
    # -------------------------------------------------
    # df.duplicated(keep=False) marca como True todas las filas que tienen otra igual.
    # keep=False significa que marca todas las copias, no solo una.
    # .sum() cuenta cuántas filas están repetidas.
    dup_rows = df.duplicated(keep=False).sum()
    print("Filas duplicadas (exactas):", dup_rows)

    # -------------------------------------------------
    # Si se especificó una columna clave válida, analizar duplicados por esa columna
    # -------------------------------------------------
    # if clave analiza que clave no sea None
    # and (y)
    if clave and clave in df.columns:
    # clave in df.columns-- >que clave sea una columna existente dentro de las columnas del dataframe
    # si no le paso ninguna columna no va a querer encontrar duplicados por columna
    # y si me equivoco y le paso una columna que no existe en el dataframe, tampoco ingresara al if.
        # Contar cuántas filas tienen valores repetidos en esa columna
        dup_key = df[clave].duplicated(keep=False).sum()
        print(f"Duplicados por clave '{clave}':", dup_key)

        # Si existen duplicados, mostrar cuáles son los valores más repetidos
        if dup_key > 0:
            # Filtrar filas donde esa clave esté duplicada
            # df[clave].duplicated(keep=False) devuelve True donde el valor se repite
            duplicados_ordenados = (
                df[df[clave].duplicated(keep=False)][clave]
                .value_counts()                # Cuenta cuántas veces aparece cada valor
                .sort_values(ascending=False)   # Ordena de mayor a menor (más duplicados arriba)
            )

            print("\n🔁 Top valores duplicados más frecuentes:")
            # Mostrar solo los primeros 10 (los más repetidos)
            display(duplicados_ordenados.head(10))
        else:
            print(f"No se encontraron duplicados en la clave '{clave}'.")
    else:
        # Si la clave no fue pasada o no existe en el DataFrame
        if clave:
            print(f"La clave '{clave}' no existe en el DataFrame.")
        else:
            print("No se indicó una clave para analizar duplicados por columna.")
#fin de def calidad


In [ ]:
calidad(ventas, "ventas", "id_venta")

In [ ]:
calidad(clientes, "clientes", "id_cliente")

In [ ]:
calidad(marketing,"marketing","id_campanha")

## 🧹 Etapa 2: Preprocesamiento y Limpieza de Datos
**Objetivo:** Demostrar conocimiento de las técnicas de limpieza y transformación de datos.

### 4️⃣ Limpieza de datos

In [ ]:
ventas_clean = ventas.copy( deep =True )
clientes_clean = clientes.copy(deep =True)
marketing_clean = marketing.copy(deep =True)

In [ ]:
ventas_clean = ventas_clean.drop_duplicates()

In [ ]:
eliminados = ventas.shape[0] - ventas_clean.shape[0]

print(f"Se eliminaron {eliminados} filas duplicadas en ventas_clean.")

In [ ]:
def normalizar_texto(df):
    for col in df.select_dtypes(include="object").columns:
        # Se agrupan las operaciones entre paréntesis () para escribirlas en varias líneas
        # Python evalúa todo el bloque como una única expresión.
        df[col] = (
            df[col]
            .astype(str)                              # Convierte cualquier tipo a string
            # .astype(str)  → convierte todo a texto; no tiene parámetros adicionales.
            .str.strip()                               # Elimina espacios al inicio y final
            # .str.strip() no necesita argumentos; borra espacios en blanco por defecto.
            .str.replace(r"[\u200b\t\r\n]", "", regex=True)
            # .str.replace(patron, reemplazo, regex=True)
            #   patron: expresión regular que busca caracteres invisibles (\u200b, tabulaciones, saltos)
            #   reemplazo: ""  → los elimina
            #   regex=True indica que 'patron' es una expresión regular.
            .str.replace(" +", " ", regex=True)
            # reemplaza "uno o más espacios consecutivos" por un solo espacio
            .str.title()                               # Convierte a Título: "juan pérez" → "Juan Pérez"
        )

        #df[col] = df[col].astype(str).str.strip().str.replace(r"[\u200b\t\r\n]", "", regex=True).str.replace(" +", " ", regex=True).str.title()
    return df

In [ ]:
for df in [ventas_clean, clientes_clean, marketing_clean]:
    for col in df.columns:
        if "fecha" in col.lower():  # detecta columnas con la palabra "fecha"
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)

In [ ]:
for df,nombre in zip([ventas_clean, clientes_clean, marketing_clean],["ventas","cliente","marketing"]):
    print(f'df {nombre.upper()} \n')
    print(df.dtypes)
    print("-"*100)

In [ ]:
for df in [ventas_clean, clientes_clean, marketing_clean]:
    df = normalizar_texto(df)

In [ ]:
ventas_clean.head(3)

In [ ]:
if "precio" in ventas_clean.columns:
    # convierto a texto, aplico funciones de texto replace sin expresion regular, quito espacios
    ventas_clean["precio"] = ventas_clean["precio"]\
      .astype(str)\
      .str.replace("$", "", regex=False)\
      .str.replace(",", "", regex=False)\
      .str.strip()



In [ ]:
ventas_clean["precio"] = pd.to_numeric(ventas_clean["precio"], errors="coerce")

In [ ]:
if "cantidad" in ventas_clean.columns:
    ventas_clean["cantidad"] = pd.to_numeric( ventas_clean["cantidad"], errors="coerce")\
    .astype("Int64")

In [ ]:
ventas_clean = ventas_clean.dropna(axis= 0 )

In [ ]:
for df,nombre in zip([ventas_clean, clientes_clean, marketing_clean],["ventas","cliente","marketing"]):
    print(f'df {nombre.upper()} \n')
    print(df.dtypes)
    print("-"*100)

In [ ]:
ventas_clean.to_csv("/content/drive/MyDrive/datasets/ventas_clean.csv", index=False)
clientes_clean.to_csv("/content/drive/MyDrive/datasets/clientes_clean.csv", index=False)
marketing_clean.to_csv("/content/drive/MyDrive/datasets/marketing_clean.csv", index=False)

print("✅ Archivos guardados: ventas_clean.csv, clientes_clean.csv, marketing_clean.csv")

In [ ]:
def reporte_calidad_global(dfs, nombres):
    """
    Crea un resumen de calidad de varios DataFrames.

    Parámetros:
      dfs: lista de DataFrames (por ejemplo [ventas_clean, clientes_clean, marketing_clean])
      nombres: lista de nombres correspondientes (["VENTAS", "CLIENTES", "MARKETING"])
    """
    resumen = []
    #zip-->es una función incorporada de Python que une elementos de dos (o más) iterables
    # —por ejemplo, listas, tuplas o cualquier objeto iterable— en pares ordenados.
    for df, nombre in zip(dfs, nombres):
        nulos = df.isna().sum().sum()                    # Total de valores nulos, no por columnas sino total, por eso el doble sum
        duplicados = df.duplicated(keep=False).sum()     # Total de filas duplicadas
        columnas = len(df.columns)                       # Cantidad de columnas
        filas = len(df)                                  # Cantidad de registros

        resumen.append({
            "Dataset": nombre,
            "Filas": filas,
            "Columnas": columnas,
            "Nulos totales": nulos,
            "Duplicados": duplicados,
        })

    reporte = pd.DataFrame(resumen)
    #display(reporte)
    return reporte

In [ ]:
print(reporte_calidad_global([ventas, clientes, marketing], ["VENTAS Original", "CLIENTES Original", "MARKETING Original"]))
print(reporte_calidad_global([ventas_clean, clientes_clean, marketing_clean],["VENTAS Copia   ", "CLIENTES Copia   ", "MARKETING Copia   "]))

In [ ]:
def comparacion_columnas(df1, df2, nombres):
    """
    Compara las columnas de dos DataFrames y muestra las que son diferentes."""
    df =pd.concat([df1.dtypes, df2.dtypes], axis=1)
    df.columns = [nombres[0], nombres[1]]
    df['diferencia'] = np.where(df[nombres[0]] != df[nombres[1]], "Diferente", "Igual")
    return df



In [ ]:
comparacion_columnas(ventas, ventas_clean, ["ventas", "ventas_clean"])

In [ ]:
comparacion_columnas(clientes, clientes_clean, ["clientes", "clientes_clean"])

In [ ]:
comparacion_columnas(marketing, marketing_clean, ["marketing", "marketing_clean"])

### 5️⃣ Transformación de datos

In [ ]:
# TODO: Aplicar filtros y transformaciones para crear una tabla de ventas
# que muestre solo los productos con alto rendimiento. calcular el percentil 80
# y filtrar los productos que superen ese umbral en ventas.
#quantile(0.8)
# Sugerencia: usar .query() o condiciones con operadores lógicos.

2. Transformación de Datos: Aplicar filtros y transformaciones para crear una
tabla de ventas que muestre solo los productos con alto rendimiento.
3. Agregación: Resumir las ventas por categoría de producto y analizar los
ingresos generados.
4. Integración de Datos: Combinar los sets de datos de ventas y marketing
para obtener una visión más amplia de las tendencias.

In [ ]:
ventas_clean.head(3)

In [ ]:
ventas_clean['total'] = ventas_clean['precio'] * ventas_clean['cantidad']
ventas_clean.head(3)

### 6️⃣ Agregación

In [ ]:
resumen_prod = ventas_clean.groupby(by='producto', as_index=False)\
  .agg(ingreso_total=('total', 'sum'), unidades=('cantidad', 'sum'), precio_promedio=('precio', 'mean'), registros=('total', 'size'))\
  .sort_values(by='ingreso_total', ascending=False)

resumen_prod.head(3)

In [ ]:
cantidades_percentil_80 = resumen_prod['unidades'].quantile(0.8)
print(f"El percentil 80 de las cantidades es: {cantidades_percentil_80}")

In [ ]:
ingresos_percentil_80 = resumen_prod['ingreso_total'].quantile(0.8)
print(f"El percentil 80 del total de ventas es: {ingresos_percentil_80:.2f}")

In [ ]:
resumen_prod_alto_rendimiento = resumen_prod[resumen_prod['ingreso_total'] > ingresos_percentil_80].sort_values(by='ingreso_total', ascending=False)
resumen_prod_alto_rendimiento.head()

In [ ]:
ingresos_superiores = resumen_prod_alto_rendimiento['ingreso_total'].sum()
total_ingresos = resumen_prod['ingreso_total'].sum()
porcentaje_ingresos = (ingresos_superiores / total_ingresos) * 100
print(f"El porcentaje de ingresos superiores al percentil 80 es: {porcentaje_ingresos:.2f}%")


In [ ]:
cantidad_productos = resumen_prod.shape[0]
print(f"La cantidad total de productos es: {cantidad_productos}")

In [ ]:
cantidad_productos_alto_rendimiento = resumen_prod_alto_rendimiento.shape[0]
print(f"La cantidad de productos con ingresos superiores al percentil 80 es: {cantidad_productos_alto_rendimiento}")
print(f"El porcentaje de productos con ingresos superiores al percentil 80 es: {(cantidad_productos_alto_rendimiento/cantidad_productos)*100:.2f}%")

In [ ]:
resumen_prod['total_acumulado']= resumen_prod['ingreso_total'].cumsum()
resumen_prod['total_general'] = resumen_prod['ingreso_total'].sum()
resumen_prod['porcentaje_acumulado'] = ((resumen_prod['total_acumulado'] / resumen_prod['total_general']) * 100).round(2)
resumen_prod.head(30)

In [ ]:
mask = resumen_prod['porcentaje_acumulado'] < 80

In [ ]:
cantidad_productos_alto_rendimiento_2 = resumen_prod[mask].shape[0]

In [ ]:
print(f"El 80 % de los ingresos es generado por : {(cantidad_productos_alto_rendimiento_2/cantidad_productos)*100:.2f} % de los productos")

### 7️⃣ Integración de datos, opcional, NO OBLIGATORIO

In [ ]:
# TODO: Combinar los sets de datos de ventas y marketing para obtener una visión más amplia de las tendencias.
# Sugerencia: usar pd.merge() especificando la clave común entre ambos DataFrames.
# Documentar cualquier observación relevante sobre la combinación de datos.
